In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_ollama import ChatOllama

model = ChatOllama(model="llama3.1:8b", temperature=0)

In [3]:
from langchain.tools import tool, ToolRuntime

@tool
def read_email(runtime: ToolRuntime) -> str:
    """Read an email from the given address."""
    # take email from state
    return runtime.state["email"]

@tool
def send_email(body: str) -> str:
    """Send an email to the given address with the given subject and body."""
    # fake email sending
    return f"Email sent"

In [4]:
from langchain.agents import create_agent, AgentState
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import HumanInTheLoopMiddleware

class EmailState(AgentState):
    email: str

agent = create_agent(
    model=model,
    tools=[read_email, send_email],
    state_schema=EmailState,
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "read_email": False,
                "send_email": True,
            },
            description_prefix="Tool execution requires approval",
        ),
    ],
)

In [5]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {
        "messages": [HumanMessage(content="Please read my email and send a response.")],
        "email": "Hi Seán, I'm going to be late for our meeting tomorrow. Can we reschedule? Best, John."
    },
    config=config
)

In [6]:
from pprint import pprint

pprint(response)

{'__interrupt__': [Interrupt(value={'action_requests': [{'args': {'body': 'Hi '
                                                                          'Seán, '
                                                                          'I '
                                                                          'apologize '
                                                                          'but '
                                                                          'I '
                                                                          'will '
                                                                          'be '
                                                                          'late '
                                                                          'for '
                                                                          'our '
                                                                          'meeting '
                    

In [7]:
print(response['__interrupt__'])

[Interrupt(value={'action_requests': [{'name': 'send_email', 'args': {'body': 'Hi Seán, I apologize but I will be late for our meeting tomorrow. Would it be possible to reschedule?', 'recipient': 'Seán', 'subject': 'Rescheduling Meeting'}, 'description': "Tool execution requires approval\n\nTool: send_email\nArgs: {'body': 'Hi Seán, I apologize but I will be late for our meeting tomorrow. Would it be possible to reschedule?', 'recipient': 'Seán', 'subject': 'Rescheduling Meeting'}"}], 'review_configs': [{'action_name': 'send_email', 'allowed_decisions': ['approve', 'edit', 'reject']}]}, id='3de8a93ef7076a0fa7dad3f8158ca305')]


In [8]:
# Access just the 'body' argument from the tool call
print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])

Hi Seán, I apologize but I will be late for our meeting tomorrow. Would it be possible to reschedule?


## Approve

In [9]:
from langgraph.types import Command

response = agent.invoke(
    Command( 
        resume={"decisions": [{"type": "approve"}]}
    ), 
    config=config # Same thread ID to resume the paused conversation
)

pprint(response)

{'email': "Hi Seán, I'm going to be late for our meeting tomorrow. Can we "
          'reschedule? Best, John.',
 'messages': [HumanMessage(content='Please read my email and send a response.', additional_kwargs={}, response_metadata={}, id='6bda9ed4-8609-4988-bb3b-77a41f7f3463'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2026-01-10T21:09:43.131265669Z', 'done': True, 'done_reason': 'stop', 'total_duration': 59799175411, 'load_duration': 21148624674, 'prompt_eval_count': 200, 'prompt_eval_duration': 24510194479, 'eval_count': 48, 'eval_duration': 14007734843, 'logprobs': None, 'model_name': 'llama3.1:8b', 'model_provider': 'ollama'}, id='lc_run--019ba9bd-70be-7b53-bc3a-670c71250b68-0', tool_calls=[{'name': 'read_email', 'args': {}, 'id': '21be9c07-fe3c-4daa-be00-f64b674c7cf6', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 200, 'output_tokens': 48, 'total_tokens': 248}),
            

## Reject

In [10]:
response = agent.invoke(
    Command(        
        resume={
            "decisions": [
                {
                    "type": "reject",
                    # An explanation of why the request was rejected
                    "message": "No please sign off - Your merciful leader, Seán."
                }
            ]
        }
    ), 
    config=config # Same thread ID to resume the paused conversation
    )   

pprint(response)

{'email': "Hi Seán, I'm going to be late for our meeting tomorrow. Can we "
          'reschedule? Best, John.',
 'messages': [HumanMessage(content='Please read my email and send a response.', additional_kwargs={}, response_metadata={}, id='6bda9ed4-8609-4988-bb3b-77a41f7f3463'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2026-01-10T21:09:43.131265669Z', 'done': True, 'done_reason': 'stop', 'total_duration': 59799175411, 'load_duration': 21148624674, 'prompt_eval_count': 200, 'prompt_eval_duration': 24510194479, 'eval_count': 48, 'eval_duration': 14007734843, 'logprobs': None, 'model_name': 'llama3.1:8b', 'model_provider': 'ollama'}, id='lc_run--019ba9bd-70be-7b53-bc3a-670c71250b68-0', tool_calls=[{'name': 'read_email', 'args': {}, 'id': '21be9c07-fe3c-4daa-be00-f64b674c7cf6', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 200, 'output_tokens': 48, 'total_tokens': 248}),
            

In [11]:
response['messages'][-3].tool_calls[0]['args']['body']

'Hi Seán, I apologize but I will be late for our meeting tomorrow. Would it be possible to reschedule?'

## Edit

In [12]:
response = agent.invoke(
    Command(        
        resume={
            "decisions": [
                {
                    "type": "edit",
                    # Edited action with tool name and args
                    "edited_action": {
                        # Tool name to call.
                        # Will usually be the same as the original action.
                        "name": "send_email",
                        # Arguments to pass to the tool.
                        "args": {"body": "This is the last straw, you're fired!"},
                    }
                }
            ]
        }
    ), 
    config=config # Same thread ID to resume the paused conversation
    )   

pprint(response)

{'email': "Hi Seán, I'm going to be late for our meeting tomorrow. Can we "
          'reschedule? Best, John.',
 'messages': [HumanMessage(content='Please read my email and send a response.', additional_kwargs={}, response_metadata={}, id='6bda9ed4-8609-4988-bb3b-77a41f7f3463'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2026-01-10T21:09:43.131265669Z', 'done': True, 'done_reason': 'stop', 'total_duration': 59799175411, 'load_duration': 21148624674, 'prompt_eval_count': 200, 'prompt_eval_duration': 24510194479, 'eval_count': 48, 'eval_duration': 14007734843, 'logprobs': None, 'model_name': 'llama3.1:8b', 'model_provider': 'ollama'}, id='lc_run--019ba9bd-70be-7b53-bc3a-670c71250b68-0', tool_calls=[{'name': 'read_email', 'args': {}, 'id': '21be9c07-fe3c-4daa-be00-f64b674c7cf6', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 200, 'output_tokens': 48, 'total_tokens': 248}),
            